In [1]:
import numpy
import pandas

import gensim
from gensim import corpora

from pythainlp.tokenize import word_tokenize
import pythainlp

import re

### Load Corpus

In [2]:
file_directory = '../data/facebook/1/ผู้บริโภค - AllCafe.txt'
corpus = []
labels = []
    
f = open(file_directory, 'r')
data = f.read().splitlines()
f.close()
    
start = False
num = -2

for line in data:
    if line.startswith('comment'):
        num += 1
        if num < 0:
            continue

        comment = ''.join(x for x in line.split(':')[2:])
        corpus.insert(num, comment)
        labels.insert(num, line.split(':')[1])
        start = True

    elif start:
        corpus[num] += line

In [3]:
len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

Total documents 1091
5 clusters


### Preprocess Corpus

#### Tokenize Corpus

In [4]:
def clean(doc):
    while True:
        new_doc = re.sub('[^\u0E00-\u0E7F]+', '', doc)
        if doc == new_doc:
            break
        else:
            doc = new_doc
    return doc

tokenized_corpus = []
for doc in corpus:
    cleaned_doc = clean(doc)
    tokenized_corpus.append(word_tokenize(cleaned_doc, engine='deepcut'))

Using TensorFlow backend.


#### Remove Words

In [28]:
dictionary = corpora.Dictionary(tokenized_corpus)
print(dictionary.__len__(), 'words')

print('filter frequent', appear_rate * len_corpus)
appear_rate = 0.95
dictionary.filter_extremes(no_above=len_corpus * appear_rate)

short_words = []
for word in dictionary.keys():
    if len(dictionary[word]) <= 1:
        short_words.append(word)
dictionary.filter_tokens(bad_ids=short_words)

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)

print(dictionary.__len__(), 'words')

2655 words
filter frequent 1036.45
283 words


In [26]:
# bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_corpus]
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]
print([dictionary[id] for id in idx_corpus[650] if id >= 0])
print(tokenized_corpus[650])

# for doc in idx_corpus:
#     print([dictionary[id] for id in doc if id >= 0])

['พนักงาน', 'ทำ', 'หน้า', 'ชอบ', 'คุย', 'สนใจ', 'ลูกค้า']
['พนักงาน', 'ทำ', 'หน้า', 'ส้นตีน', 'ชอบ', 'กระแทก', 'ของ', 'ห่วง', 'คุย', 'ไม่', 'สนใจ', 'ลูกค้า']


In [7]:
unique_words = [dictionary[id] for id in range(len(dictionary.keys()))]

array = numpy.zeros((len_corpus, len(unique_words)), dtype=int)
for i, vector in enumerate(idx_corpus):
    for id in vector:
        if id >= 0:
            array[i, id] += 1

word_vectors = pandas.DataFrame(array, columns=unique_words, dtype=int)
word_vectors.head()

,กด,กาแฟ,ครึ่ง,ดีด,บาท,ลด,แก้ว,โดน,คน,ตีน,...,ข้าม,ชงกาแฟ,ไหร่,กรอก,บางสาขา,เปน,เเก้ว,ทาน,ขำ,เม้น
0,1,1,1,2,1,2,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
cluster_num = len(clusters)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=cluster_num).fit(word_vectors)
predicted_labels = kmeans.labels_

In [9]:
result = pandas.DataFrame()
result['comment'] = corpus
result['label'] = labels
result['predicted_label'] = predicted_labels
result.head()

,comment,label,predicted_label
0,ลดความดีดของกาแฟลงหน่อยครับ เอ่กาแฟกดแก้ว24บาท...,a,2
1,คนที่บอกว่าพนักงานทำหน้าเหมือนส้นตีนแสดงว่าคุณ...,b,1
2,สงสารพนักงานเซเว่นครับ เงินเดือนไม่มาก หน้าที่...,b,1
3,ชงให้มันเร็วๆหน่อยบางทีกูรีบไปทำงาน มัวแต่คุยก...,b,1
4,ไม่ฝากไรมากค่ะ สงสารพนักงาน มึงจะให้เค้าเป็นทุ...,b,2


In [10]:
label_count = numpy.unique(predicted_labels, return_counts=True) 
print(label_count)

for cluster in clusters:
    print('\t' + cluster, end='')
print('\tpercent')

for label in range(cluster_num):
    print(str(label) + '  |', end='')
    
    num_max = 0
    for cluster in clusters:
        loc = result[(result['label'] == cluster) & (result['predicted_label'] == label)]
        if len(loc) > num_max:
            num_max = len(loc)
        print('\t' + str(len(loc)), end='')
    
    print('\t' + str(num_max / label_count[1][label]))

(array([0, 1, 2, 3, 4], dtype=int32), array([ 71,  85, 779,  63,  93]))
	d	a	c	e	b	percent
0  |	0	18	7	0	46	0.647887323943662
1  |	0	11	10	0	64	0.7529411764705882
2  |	39	357	127	58	198	0.4582798459563543
3  |	0	51	1	0	11	0.8095238095238095
4  |	0	61	12	1	19	0.6559139784946236


In [11]:
for doc in result[result['predicted_label'] == 1]['comment'].values:
    print('[', doc, ']')

[ คนที่บอกว่าพนักงานทำหน้าเหมือนส้นตีนแสดงว่าคุณไม่ใช่กลุ่ม  ของเขาครับ ]
[ สงสารพนักงานเซเว่นครับ เงินเดือนไม่มาก หน้าที่เกินเงินเดือนครับ 5555 ขายของ ขายกาแก เติมเงิน ล่าสุดกำลังจะรับฝากเงินให้ธออมสิน 5555 ]
[ ชงให้มันเร็วๆหน่อยบางทีกูรีบไปทำงาน มัวแต่คุยกันอยู่ได้ชิบหาย ]
[ ข้อดีข้อเสียแยกออกไปตามสาขา พนักงานดีและพนักงานไม่ดีก็มีแต่ละสาขา อีกหลายพันสาขาที่เรายังไม่เคยสัมผัส บางทีพนักงานที่ทำดีก็อาจจะกำลังอ่านคอมเม้นพวกคุณแล้วแอบน้อยใจอยู่ก็ได้ ทั้งที่ทำดีแล้วแต่กลับยังไม่ถูกใจ คนทำดีก็ทำไป คนทำไม่ดีนี่ก็ดังเอ๊าดังเอา อยากให้แยกแยะคะ สาขาไหนระบุไปเลย เชื่อว่าพนักงานหลายคนทำเต็มที่และทำด้วยใจถึงขนาดกลับบ้านไปนอนละเมอก็มี จากใจดวงนึง ]
[ มึงควรจะจ้างพนักงานชงประจำอย่างเดียวซัก 1 ตำแหน่ง ไม่ใช่ให้เด็ก 7 แม่งทำทุกอย่างจนหน้าเป็นตูดแล้วก็โดนคนด่า ]
[ สงสารพนักงาน เซเว่น ครับ คือไอเรื่องกาแฟ อ่ะ ก็พอรับได้ ในราคา 30 40 บาท  เพราะพนักงานก็ใช่ว่า ะจ้าวมาเน้นในการทำกาแฟ โดยเฉพาะ ทำหลายอย่าง ลูกค้าเยอะ ละต้องรีบมาชงกาแฟอีก อาจมีพลาดหลงลืมสูตร ผิดเพี้ยนกันไปบ้างได้  สำหรับผมพอรับได้กับรสชาติ ใน

In [14]:
for row_num in result[result['predicted_label'] == 1]['comment'].index:
    print(row_num, [dictionary[id] for id in idx_corpus[row_num] if id >= 0])

1 ['คน', 'พนักงาน', 'ทำ', 'หน้า', 'เหมือน', 'ส้น', 'ตีน']
2 ['สงสาร', 'พนักงาน', 'เงิน', 'เดือน', 'หน้าที่', 'เงิน', 'เดือน', 'ขาย', 'ขาย', 'เติม', 'เงิน', 'ฝาก', 'เงิน']
3 ['ชง', 'รีบ', 'ทำ', 'งาน', 'มัว', 'คุย', 'ชิบ', 'หาย']
12 ['ข้อ', 'ดี', 'ข้อ', 'สาขา', 'พนักงาน', 'ดี', 'พนักงาน', 'ดี', 'สาขา', 'สาขา', 'พนักงาน', 'ทำ', 'ดี', 'อ่าน', 'คอมเม้น', 'ทำ', 'ดี', 'คน', 'ทำ', 'ดี', 'ทำ', 'คน', 'ทำ', 'ดี', 'พนักงาน', 'คน', 'ทำ', 'ทำ', 'ใจ', 'ขนาด', 'บ้าน', 'นอน', 'นึง']
16 ['พนักงาน', 'ชง', 'ประจำ', 'ซัก', 'เด็ก', 'แม่ง', 'ทำ', 'หน้า', 'โดน', 'คน', 'ด่า']
19 ['สงสาร', 'พนักงาน', 'เซเว่น', 'เรื่อง', 'กาแฟ', 'อ่ะ', 'ราคา', 'บาท', 'พนักงาน', 'ทำ', 'กาแฟ', 'ทำ', 'ลูกค้า', 'รีบ', 'ชง', 'กาแฟ', 'สูตร', 'สำหรับ', 'ผม', 'รสชาติ', 'ราคา', 'บาท']
21 ['ทำ', 'งาน', 'ชีวิต', 'งาน', 'ทำ', 'ใส่', 'ตอน', 'รู้', 'เหมือน', 'เดิม']
25 ['รสชาติ', 'พนักงาน', 'เรียน', 'สัก', 'พนักงาน', 'คน', 'งง', 'ดี']
27 ['เกี่ยง', 'ทำ', 'ทำ', 'หน้า']
35 ['ผม', 'ทำ', 'งาน', 'เซเว่น', 'ทำ', 'สูตร', 'ทำ', 'สูตร', 'แปลก', 'สั่ง'

In [27]:
for doc in corpus:
    if 'ส้นตีน' in doc:
        print(doc)

คนที่บอกว่าพนักงานทำหน้าเหมือนส้นตีนแสดงว่าคุณไม่ใช่กลุ่ม  ของเขาครับ
คนที่บอกพนักงานชงกาแฟหน้าหยั่งส้นตีนน่ะเข้าใจเขาบ้าง เด็กแถวบ้านกูเมื่อก่อนมันทำงานร้านนี่คอยคิดตังค์ตามปกติหน้าอย่างแบ๊ว พอร้านแม่งเอาไอ้กาแฟนี่มาลงแล้วบังคับให้เด็กต้องหัดชงขายเท่านั้นแหละ หน้าเหี้ยในทันที
พนง บางสาขาชงกาแฟหน้าอย่างกับส้นตีน ไม่อยากขายไม่อยากชงก็ลาออกไปเหอะ
สั่งกาแฟทีไร ทำไมต้องทำหน้าเหมือนส้นตีนทู๊กกกกกกที กูก็ไม่เข้าใจ
พนักงาน คัดคุณภาพดีๆ แต่ละคนต้อนรับหน้ายังกับหีโดนสิบล้อทับ พูดจายังกะเขมรเรียนภาษาไทย อีสัสกวนส้นตีน เกือบทุกสาขาอะ มองคนหัวจรดตีน
พนักงานทำหน้าเหมือนส้นตีน ทะเลาะกับผัวแม่งทุกวัน
พนักงานทำหน้าส้นตีน ชอบกระแทกของ ห่วงคุย ไม่สนใจลูกค้า
พนักงานคับ อาการส้นตีนมีบ่อย
หงึกเป็นส้นตีนจริงๆค่ะหน้า เห็นแล้วหงุดหงิด
